In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
# Load and preprocess the MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1)) 
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Add noise to the images
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [3]:
from tensorflow.keras.models import load_model
base_model = load_model('final_feature.h5')
base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
class Model:
    def __init__(self,model):
        self.model=model
    def show(self):
        path_cm = './confusion_matrix.png'
        cm = Image.open(path_cm)
        print("Model Evaluation Metrics:")
        display(cm)
        path_results = './result_model.png'
        results = Image.open(path_results)
        print("\nThis model was fine-tuned(VGG16) on Architectural Heritage Elements data.")
        print("Trained on 3000 images across 10 classes.")
        print("******************")
        print("Results on testing data:")
        display(results)

In [6]:
class KL_LOSS:
    def _init__(self):
        self.p=None
        self.q=None
    def hist(self,f1,f2):
        print("Finding Probability Distribution of Features.")
        self.p, _ = np.histogram(f1, bins=256, range=(0, 256), density=True)
        self.q, _ = np.histogram(f2, bins=256, range=(0, 256), density=True)
        self.p = self.p[:256]
        self.q = self.q[:256]
        x_range = np.arange(256)
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        axes[0, 0].plot(x_range, self.p, color='blue')
        axes[0, 0].set_title('Density of hist1')
        axes[0, 0].set_xlabel('Pixel Value')
        axes[0, 0].set_ylabel('Density')

        axes[0, 1].hist(np.arange(256), bins=256, weights=self.p, color='blue')
        axes[0, 1].set_title('Histogram of hist1')
        axes[0, 1].set_xlabel('Pixel Value')
        axes[0, 1].set_ylabel('Frequency')

        axes[1, 0].plot(x_range, self.q, color='red')
        axes[1, 0].set_title('Density of hist2')

        axes[1, 0].set_xlabel('Pixel Value')
        axes[1, 0].set_ylabel('Density')

        axes[1, 1].hist(np.arange(256), bins=256, weights=self.q, color='red')
        axes[1, 1].set_title('Histogram of hist2')
        axes[1, 1].set_xlabel('Pixel Value')
        axes[1, 1].set_ylabel('Frequency')
        plt.tight_layout()
        plt.show()
        print("Done.")
        
        
    def kl_Loss(self,epsilon=1e-10):
        try:
            print("Finding KL Loss.")
            p=self.p
            q=self.q
            pq=sum(p[i] * log2((p[i] + epsilon) / (q[i] + epsilon)) for i in range(len(p)))
            qp=sum(q[i] * log2((q[i] + epsilon) / (p[i] + epsilon)) for i in range(len(q)))
            p='image1'
            q='image2'
            print(" {},differs from another distribution, {}: {}\n\n".format(p,q,pq))
            print(" {},differs from another distribution, {}: {}\n\n".format(q,p,qp))
            print("KL Loss:",(pq+qp))
        except Exception as e:
            print("First call function hist.")
            print(e)

In [7]:
class ContentLoss(Model,KL_LOSS):
    def __init__(self,image1,image2,model):
        self.model=model
        super().__init__(self.model)
        self.image1=image1
        self.image2=image2
        self.__list_fm1=None
        self.__list_fm2=None
        self.euclidean=[]
        self.cosine=[]
        self.contentloss=[]
        self.kl_loss = KL_LOSS()
        
    def preprocess(self):
        array1 = np.array(self.image1)
        array2 = np.array(self.image2)
        fig, axes = plt.subplots(1, 2)
        axes[0].imshow(image1)
        axes[0].set_title('Image 1')
        axes[1].imshow(image2)
        axes[1].set_title('Image 2')
        plt.show()
        
    def generate_features(self):
        image1 = np.expand_dims(self.image1, axis=0)
        image2 = np.expand_dims(self.image2, axis=0)
        self.__list_fm1 = self.model.predict(image1)
        self.__list_fm2 = self.model.predict(image2)
        print("done.")
        
        
    def sim_euc(self):
        for i in range(len(self.__list_fm1)):
            flat_features_image1 = self.__list_fm1[i].flatten()
            flat_features_image2 = self.__list_fm2[i].flatten()
            euclidean_distance = euclidean_distances([flat_features_image1], [flat_features_image2])
            cosine_similarity_score = cosine_similarity([flat_features_image1], [flat_features_image2])
            self.euclidean.append(euclidean_distance)
            self.cosine.append(cosine_similarity_score)
        print("done.")
            
    def Contentloss(self):
        print("inform:........preprocessing  image")
        self.preprocess()
        print("Inform:........Finding features map of 2 given images.")
        self.generate_features()
        print("Done..\n")
        print("Finding distance.......")
        self.sim_euc()
        print("Finding Content loss.............\n")
        print("**********************************")
        for i in range(len(self.euclidean)):
            print(f"Euclidean Distance for feature map {i+1}: {self.euclidean[i]}")
            print(f"Cosine Similarity for feature map {i+1}: {self.cosine[i]}\n")
            print("****************************************************************")
        print("************************************")
        print("content loss: Giving 100 times to 5th and 10 times to 4th...",100*self.euclidean[4]+10*self.euclidean[3])
    
    def kl(self):
        self.kl_loss.hist(self.__list_fm1[4],self.__list_fm2[4])
#         self.kl_loss.hist(self.image1,self.image2)
        self.kl_loss.kl_Loss()
    

In [8]:
# Define the Autoencoder architecture
def autoencoder(input_shape):
    # Encoder
    encoder_input = layers.Input(shape=input_shape, name='encoder_input')
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder_model = models.Model(encoder_input, decoded, name='autoencoder')

    return autoencoder_model

In [9]:
input_shape = (28, 28, 1)  
# Create the autoencoder model
model = autoencoder(input_shape)
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy')

In [10]:
model.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          

In [11]:
epochs = 1
batch_size = 128
history = model.fit(x_train, x_train_nosiy, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(x_test_noisy, x_test))

NameError: name 'x_train_nosiy' is not defined

In [ ]:
# Plot training loss and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
decoded_images = model.predict(x_test_noisy)
n = 10 
plt.figure(figsize=(20, 6))
for i in range(n):
    # Original Images
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Noisy Images
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Denoised Images
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(decoded_images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()